# Get Domain

**Author:** Andrew Loeppky (Lots of code stolen from Jamie Byer)

**Project:** Land-surface-atmosphere coupling - CMIP6 intercomparison 

This notebook is meant to acquire a dataset from the CMIP6 data library, chop out a pre-specified spatial slice (between coordinates specified by user), and save the dataset in Zarr format. Also adds a 3d pressure field variable, converting from surface pressure and sigma values $ap$ and $b$

## Helpful Docs

https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q/edit#

https://towardsdatascience.com/a-quick-introduction-to-cmip6-e017127a49d3

https://pcmdi.llnl.gov/CMIP6/Guide/dataUsers.html

http://proj.badc.rl.ac.uk/svn/exarch/CMIP6dreq/tags/latest/dreqPy/docs/CMIP6_MIP_tables.xlsx

https://esgf-node.llnl.gov/search/cmip6/

In [1]:
# Attributes of the model we want to analyze (put in csv later)
source_id = 'ACCESS-CM2'
source_id = 'GFDL-ESM4'
experiment_id = 'piControl'
table_id = 'Amon'

# Domain we wish to study
lats = (10, 20) # lat min, lat max
lons = (20, 29) # lon min, lon max
times = ()
ceil = 500 # top of domain, hPa

# variables of interest
fields_of_interest = ("ps",  # surface pressure
                      "cl",  # cloud fraction
                      "ta",  # air temperature
                      "ts",  # surface temperature
                      "hus", # specific humidity
                      "hfls", # Surface Upward Latent Heat Flux
                      "hfss", # Surface Upward Sensible Heat Flux
                      "rlds",  # surface downwelling longwave
                      "rlus",  # surface upwelling longwave
                      "rsds", # downwelling short wave
                      "rsus", # upwelling short wave
                      "hurs",  # near surface RH
                      "pr", # precipitation, all phases
                      "evspsbl", # evaporation, sublimation, transpiration
                      "wap"  # omega (subsidence rate in pressure coords)
                     )

In [2]:
# hourly data (put in csv with monthly control set)
#source_id = 'GFDL-ESM4'
#experiment_id = 'piControl'
#table_id = 'CF3hr'

#lats = (10, 20) # lat min, lat max
#lons = (20, 29) # lon min, lon max
#ceil = 500 # top of domain, hPa

# variables of interest
#fields_of_interest = ("ps",  # surface pressure
#                      "ta",)  # air temperature
                     # "ts",  # surface temperature
                     # "hus", # specific humidity
                     # "hfls", # Surface Upward Latent Heat Flux
                     # "hfss", # Surface Upward Sensible Heat Flux
                     # "rlds",  # surface downwelling longwave
                     # "rlus",  # surface upwelling longwave
                     # "rsds", # downwelling short wave
                     # "rsus", # upwelling short wave
                     # "hurs",  # near surface RH
                     # "pr", # precipitation, all phases
                     # "evspsbl", # evaporation, sublimation, transpiration
                     #)

In [3]:
import xarray as xr
import pooch
import pandas as pd
import fsspec
from pathlib import Path
import time
import numpy as np
import json

In [4]:
#get esm datastore
odie = pooch.create(
    path="./.cache",
    base_url="https://storage.googleapis.com/cmip6/",
    registry={
        "pangeo-cmip6.csv": None
    },
)
file_path = odie.fetch("pangeo-cmip6.csv")
df_og = pd.read_csv(file_path)

In [5]:
def fetch_var_exact(the_dict,df_og):
    the_keys = list(the_dict.keys())
    #print(the_keys)
    key0 = the_keys[0]
    #print(key0)
    #print(the_dict[key0])
    hit0 = df_og[key0] == the_dict[key0]
    if len(the_keys) > 1:
        hitnew = hit0
        for key in the_keys[1:]:
            hit = df_og[key] == the_dict[key]
            hitnew = np.logical_and(hitnew,hit)
            #print("total hits: ",np.sum(hitnew))
    else:
        hitnew = hit0
    df_result = df_og[hitnew]
    return df_result

In [6]:
def get_field(variable_id, 
              df,
              source_id=source_id,
              experiment_id=experiment_id,
              table_id=table_id):
    """
    extracts a single variable field from the model
    """

    var_dict = dict(source_id = source_id, variable_id = variable_id,
                    experiment_id = experiment_id, table_id = table_id)
    local_var = fetch_var_exact(var_dict, df)
    zstore_url = local_var['zstore'].array[0]
    the_mapper=fsspec.get_mapper(zstore_url)
    local_var = xr.open_zarr(the_mapper, consolidated=True)
    return local_var

In [7]:
def trim_field(df, lat, lon):
    """
    cuts out a specified domain from an xarrray field
    
    lat = (minlat, maxlat)
    lon = (minlon, maxlon)
    """
    new_field = df.sel(lat=slice(lat[0],lat[1]), lon=slice(lon[0],lon[1]))
    return new_field

## Create one big dataset to represent our domain

In [8]:
# grab all fields of interest and combine
my_fields = [get_field(field, df_og) for field in fields_of_interest]
small_fields = [trim_field(field, lats, lons) for field in my_fields]
my_ds = xr.combine_by_coords(small_fields, compat="broadcast_equals", combine_attrs="drop_conflicts")

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:423: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of 

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:423: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:423: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(

In [9]:
# add pressure field, convert from sigma pressure
def press_from_sigma(ds):
    """
    takes in an xarray Dataset with variables:
        "ps" - surface pressure (Pa)
        "ap" - sigma pressure coordinate
        "b"  - sigma pressure coordinate

    returns the Dataset with a variable "p", the full pressure (Pa)
    """
    ds["p"] = ds.ap + ds.b * ds.ps
    return ds

In [10]:
def p_lcl(ds):
    """
    takes in an xarray Dataset with variables:
        "p" - pressure
        "ts" - surface (2m) air temperature
        "hurs" - near surface relative humidity
        "hus" - specific humidity
        
    horizontally averaged over the domain of the Dataset
    """
    pass

In [11]:
#mean_tsurf = my_ds.ts.mean(dim=("lat", "lon"))
#mean_psurf = my_ds.ps.mean(dim=("lat", "lon"))

In [12]:
# apply functions defined above to the field
#my_ds = press_from_sigma(my_ds)

In [15]:
my_ds

<xarray.Dataset>
Dimensions:    (lev: 85, bnds: 2, time: 6000, lat: 8, lon: 4, plev: 19)
Coordinates:
    b_bnds     (lev, bnds) float64 dask.array<chunksize=(85, 2), meta=np.ndarray>
  * lat        (lat) float64 10.62 11.88 13.12 14.38 15.62 16.88 18.12 19.38
    lat_bnds   (lat, bnds) float64 10.0 11.25 11.25 12.5 ... 18.75 18.75 20.0
  * lev        (lev) float64 20.0 53.33 100.0 ... 7.378e+04 7.91e+04 8.5e+04
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(85, 2), meta=np.ndarray>
  * lon        (lon) float64 21.56 23.44 25.31 27.19
    lon_bnds   (lon, bnds) float64 20.62 22.5 22.5 24.38 24.38 26.25 26.25 28.12
  * time       (time) object 0950-01-16 12:00:00 ... 1449-12-16 12:00:00
    time_bnds  (time, bnds) object 0950-01-01 00:00:00 ... 1450-01-01 00:00:00
    height     float64 ...
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
    plev_bnds  (plev, bnds) float64 1.038e+05 9.625e+04 ... 300.0 -100.0
Dimensions without coordinates: bnds
Data variables: (12/17)
    b          (lev) float64 dask.array<chunksize=(85,), meta=np.ndarray>
    cl         (time, lev, lat, lon) float32 dask.array<chunksize=(11, 85, 8, 4), meta=np.ndarray>
    orog       (lat, lon) float32 dask.array<chunksize=(8, 4), meta=np.ndarray>
    evspsbl    (time, lat, lon) float32 dask.array<chunksize=(500, 8, 4), meta=np.ndarray>
    hfls       (time, lat, lon) float32 dask.array<chunksize=(552, 8, 4), meta=np.ndarray>
    hfss       (time, lat, lon) float32 dask.array<chunksize=(562, 8, 4), meta=np.ndarray>
    ...         ...
    rlus       (time, lat, lon) float32 dask.array<chunksize=(549, 8, 4), meta=np.ndarray>
    rsds       (time, lat, lon) float32 dask.array<chunksize=(554, 8, 4), meta=np.ndarray>
    rsus       (time, lat, lon) float32 dask.array<chunksize=(573, 8, 4), meta=np.ndarray>
    ta         (time, plev, lat, lon) float32 dask.array<chunksize=(39, 19, 8, 4), meta=np.ndarray>
    ts         (time, lat, lon) float32 dask.array<chunksize=(665, 8, 4), meta=np.ndarray>
    wap        (time, plev, lat, lon) float32 dask.array<chunksize=(31, 19, 8, 4), meta=np.ndarray>
Attributes: (12/43)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  346615.0
    cmor_version:           3.4.0
    ...                     ...
    table_id:               Amon
    table_info:             Creation Date:(30 April 2019) MD5:e14f55f257cceaf...
    title:                  ACCESS-CM2 output prepared for CMIP6
    variant_label:          r1i1p1f1
    version:                v20191112
    version_id:             v20191112

In [14]:
print(f"""Fetched domain:
          {source_id = }
          {experiment_id = }
          {table_id = }
          {lats = }
          {lons = }
          dataset name: my_ds (xarray Dataset)""")

Fetched domain:
          source_id = 'ACCESS-CM2'
          experiment_id = 'piControl'
          table_id = 'Amon'
          lats = (10, 20)
          lons = (20, 29)
          dataset name: my_ds (xarray Dataset)
